# EFIxTCT (Cross-track ion flow)

> Abstract: Access to the 2Hz & 16Hz cross-track ion flow data derived from the Thermal Ion Imager (TII), part of the Electric Field Instrument package (EFI).
> 
> For more information about this product, see the [release notes](https://earth.esa.int/eogateway/documents/20142/37627/swarm-EFI-TII-cross-track-flow-dataset-release-notes.pdf).

In [ ]:
SERVER_URL = 'https://vires.services/ows'

In [ ]:
# Display important package versions used
%load_ext watermark
%watermark -i -v -p viresclient,pandas,xarray,matplotlib

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
# Control the HTML display of the datasets
xr.set_options(display_expand_attrs=False, display_expand_coords=True, display_expand_data=True)

from viresclient import SwarmRequest

In [ ]:
request = SwarmRequest(SERVER_URL)

## What data is available?

There are two sets of collections available, one for 2Hz and one for 16Hz, and for each there are three collections, one for each Swarm spacecraft.

In [ ]:
request.available_collections("EFI_TCT02", details=False)

In [ ]:
request.available_collections("EFI_TCT16", details=False)

In [ ]:
print(request.available_measurements("EFI_TCT02"))

In [ ]:
print(request.available_measurements("EFI_TCT16"))

As seen above, the variables available for both the 2Hz and 16Hz datasets are the same. Here is a short description for each variable:

In [ ]:
tct_vars = [
    # Satellite velocity in NEC frame
    "VsatC", "VsatE", "VsatN",
    # Geomagnetic field components derived from 1Hz product
    #  (in satellite-track coordinates)
    "Bx", "By", "Bz",
    # Electric field components derived from -VxB with along-track ion drift
    #  (in satellite-track coordinates)
    # Eh: derived from horizontal sensor
    # Ev: derived from vertical sensor
    "Ehx", "Ehy", "Ehz",
    "Evx", "Evy", "Evz",
    # Ion drift corotation signal, removed from ion drift & electric field
    #  (in satellite-track coordinates)
    "Vicrx", "Vicry", "Vicrz",
    # Ion drifts along-track from vertical (..v) and horizontal (..h) TII sensor
    "Vixv", "Vixh",
    # Ion drifts cross-track (y from horizontal sensor, z from vertical sensor)
    #  (in satellite-track coordinates)
    "Viy", "Viz",
    # Random error estimates for the above
    #  (Negative value indicates no estimate available)
    "Vixv_error", "Vixh_error", "Viy_error", "Viz_error",
    # Quasi-dipole magnetic latitude and local time
    #  redundant with VirES auxiliaries, QDLat & MLT
    "Latitude_QD", "MLT_QD",
    # Refer to release notes link above for details:
    "Calibration_flags", "Quality_flags",
]

## Fetching and plotting data

For demonstration, we will fetch the 2Hz data from Swarm Alpha (`SW_EXPT_EFIA_TCT02`)

In [ ]:
start = "2018-07-17T11:00:00"
end = "2018-07-17T16:00:00"

request = SwarmRequest(SERVER_URL)
request.set_collection("SW_EXPT_EFIA_TCT02")
request.set_products(measurements=tct_vars)
data = request.get_between(start, end)


Data can be loaded as either a pandas datframe or a xarray dataset.

In [ ]:
df = data.as_dataframe()
df.head()

In [ ]:
ds = data.as_xarray()
ds

An example plot:

In [ ]:
fig, axes = plt.subplots(nrows=4, sharex=True, figsize=(10, 7))
# Plot velocities with left axis
ds.plot.scatter(x="Timestamp", y="Vixv", ax=axes[0], s=1, linewidths=0)
ds.plot.scatter(x="Timestamp", y="Vixh", ax=axes[1], s=1, linewidths=0)
ds.plot.scatter(x="Timestamp", y="Viy", ax=axes[2], s=1, linewidths=0)
ds.plot.scatter(x="Timestamp", y="Viz", ax=axes[3], s=1, linewidths=0, label="Velocities")
# Plot velocities with right axis
axes_r = [ax.twinx() for ax in axes]
ds.plot.scatter(x="Timestamp", y="Vixv_error", ax=axes_r[0], s=0.1, color="tab:orange")
ds.plot.scatter(x="Timestamp", y="Vixh_error", ax=axes_r[1], s=0.1, color="tab:orange")
ds.plot.scatter(x="Timestamp", y="Viy_error", ax=axes_r[2], s=0.1, color="tab:orange")
ds.plot.scatter(x="Timestamp", y="Viz_error", ax=axes_r[3], s=0.1, color="tab:orange")
fig.subplots_adjust(hspace=0)
# Add legend to identify each side
blue = mpl.patches.Patch(color="tab:blue", label="Velocities")
orange = mpl.patches.Patch(color="tab:orange", label="Errors")
axes[0].legend(handles=[blue, orange])
# # Generate additional ticklabels for x-axis
# Use time xticks to get dataset vars at those xticks
locx = axes[-1].get_xticks()
times = mpl.dates.num2date(locx)
times = [t.replace(tzinfo=None) for t in times]
_ds_xticks = ds.reindex({"Timestamp": times}, method="nearest")
# Build ticklabels from dataset vars
xticklabels = np.stack([
    _ds_xticks["Timestamp"].dt.strftime("%H:%M").values,
    np.round(_ds_xticks["Latitude"].values, 2).astype(str),
    np.round(_ds_xticks["Longitude"].values, 2).astype(str),
])
xticklabels = ["\n".join(row) for row in xticklabels.T]
# Add labels to first xtick
_xt0 = xticklabels[0].split("\n")
xticklabels[0] = f"Time:  {_xt0[0]}\nLat:  {_xt0[1]}\nLon: {_xt0[2]}"
axes[-1].set_xticks(axes[-1].get_xticks())
axes[-1].set_xticklabels(xticklabels)
axes[-1].set_xlabel("")
# Adjust title
title = "".join([
    f"Swarm {ds['Spacecraft'].data[0]} 2Hz ion flow, ",
    ds["Timestamp"].dt.date.data[0].isoformat(),
    f"\n{ds.attrs['Sources']}"
])
fig.suptitle(title);

Due to contamination in the instrument, great care must be taken to use these data correctly. Check the release notes and make use of the `Quality_flags` variable to identify valid data periods.

TODO: use section 3.4.1.1 to identify untrusty periods (bitx = 0) and shade them grey?